# What apps are most liked by users

- This goal of this project is to analyze what type of apps attract more users.

We will use two datasets.
1.[Google play app data](https://www.kaggle.com/lava18/google-play-store-apps) 
2.[Apple Mobile App Data](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [1]:
from csv import reader

def explore_data(dataset,start,end,rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:',len(dataset))
        print('Number of columns:', len(dataset[0]))
        print('\n')

        
def open_data(file_name):
    file = open(file_name,encoding='utf8')
    read_file = reader(file)
    dataset = list(read_file)
    return dataset
    
ios_file=open_data('AppleStore.csv')
google_file=open_data('googleplaystore.csv')

In [2]:
#explore_data(ios_file[1:],1,2,True)
explore_data(google_file[1:],1,2,True)

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13




In [3]:
print(google_file[0:2])

[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']]


In [4]:
print(google_file[10473])
print(len(google_file[10473]))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
12


In [5]:
del google_file[10473]
print(google_file[10473])
print(len(google_file[10473]))


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']
13


From the [disscussion](https://www.kaggle.com/lava18/google-play-store-apps/discussion) we can see that there may be duplicate datas. 
So I am going to make a function which finds out duplicate data entries. 

1.First I am going to find what kind of data has duplicate entries
- i will try to find apps with dupliactes entries

In [6]:
print(google_file[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [7]:
def find_duplicates(dataset,header=True):
    if header:
        apps_data= dataset[1:]
    else:
        apps_data= dataset
    duplicate = []
    unique_app = []
    for row in apps_data:
        if row[0] in unique_app:
            duplicate.append(row[0])
        else:
            unique_app.append(row[0])
    
    print('Number of duplicate app: ',len(duplicate))
    print('Example of duplicate app:', duplicate[:15])
    return duplicate
            

Here we will first get all the duplicates app by the `find_duplicates` function
the function return alist the name of the app that has duplicate entries.

-> `find dupliactes` has two parameters. dataset and header 
-> if your dataset doesnt have header row, pass `header=False` as parameter. 
->The default value is `True` as we assume that the dataset has header while being passed as arguement 

In [8]:
GooglePlay_duplicates = find_duplicates(google_file)

Number of duplicate app:  1181
Example of duplicate app: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']
